In [13]:
! pip install geopandas
! pip install pulp
! pip install spopt
! pip install coinor-cbc
! pip install matplotlib-scalebar

ERROR: Could not find a version that satisfies the requirement coinor-cbc (from versions: none)
ERROR: No matching distribution found for coinor-cbc


In [14]:
%config InlineBackend.figure_format = "retina"
%load_ext watermark
%watermark

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: 2024-05-30T22:45:49.942642+07:00

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.24.0

Compiler    : MSC v.1938 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [15]:
import pandas as pd
import networkx as nx
import geopandas
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.lines as mlines
import matplotlib_scalebar
from matplotlib_scalebar.scalebar import ScaleBar
from shapely.geometry import Point
import time
import numpy as np
import pulp
import shapely
import spopt
from spopt.locate import PCenter, simulated_geo_points

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # ignore deprecation warning - GH pysal/spaghetti#649
    import spaghetti

%watermark -w
%watermark -iv

Watermark: 2.4.3

spopt              : 0.6.0
matplotlib         : 3.8.4
pulp               : 2.8.0
networkx           : 3.3
pandas             : 2.2.2
numpy              : 1.26.4
spaghetti          : 1.7.5.post1
matplotlib_scalebar: 0.8.1
shapely            : 2.0.4
geopandas          : 0.14.4



In [16]:
data1 = pd.read_csv('C:/Users/ASUS/Downloads/Graph/DB_Result_CSV_Rescue_Station/network_distance.csv')
# data1.head(6)
data1

,StartPoint,EndPoint,Distance,net_length
0,0,1,10.0,500.0
1,1,2,10.0,0.0
2,2,3,10.0,0.0
3,3,4,10.0,0.0
4,4,5,10.0,0.0
...,...,...,...,...
2496,49,45,40.0,0.0
2497,49,46,30.0,0.0
2498,49,47,20.0,0.0
2499,49,48,10.0,0.0


In [17]:
# Lấy các giá trị duy nhất từ cột 'StartPoint'
unique_destinations = set(data1['StartPoint'])

# In ra các giá trị duy nhất dưới dạng DataFrame
unique_destinations_df = pd.DataFrame({'StartPoint': list(unique_destinations)})
print(unique_destinations_df)

    StartPoint
0            0
1            1
2            2
3            3
4            4
5            5
6            6
7            7
8            8
9            9
10          10
11          11
12          12
13          13
14          14
15          15
16          16
17          17
18          18
19          19
20          20
21          21
22          22
23          23
24          24
25          25
26          26
27          27
28          28
29          29
30          30
31          31
32          32
33          33
34          34
35          35
36          36
37          37
38          38
39          39
40          40
41          41
42          42
43          43
44          44
45          45
46          46
47          47
48          48
49          49


In [18]:
data2 = pd.read_csv('C:/Users/ASUS/Downloads/Graph/Thuyhe_Diemven2/Thuyhe_Diemven2.csv')
# data2.head(6)
data2

,Id,XX,YY
0,0,612684.883012,1.189163e+06
1,1,612684.883012,1.189173e+06
2,2,612684.883012,1.189183e+06
3,3,612684.883012,1.189193e+06
4,4,612684.883012,1.189203e+06
...,...,...,...
9841,9841,618633.328360,1.197564e+06
9842,9842,618630.751070,1.197554e+06
9843,9843,618628.184968,1.197544e+06
9844,9844,618625.906735,1.197535e+06


In [19]:
# Gộp hai DataFrame dựa trên cột 'Id' và chỉ lấy cột 'name' từ unique_destinations_df
merged_df = pd.merge(unique_destinations_df, data2[['Id', 'XX', 'YY']], left_on='StartPoint', right_on='Id')

# In ra kết quả
print(merged_df)

    StartPoint  Id             XX            YY
0            0   0  612684.883012  1.189163e+06
1            1   1  612684.883012  1.189173e+06
2            2   2  612684.883012  1.189183e+06
3            3   3  612684.883012  1.189193e+06
4            4   4  612684.883012  1.189203e+06
5            5   5  612684.883012  1.189213e+06
6            6   6  612684.883012  1.189223e+06
7            7   7  612684.883012  1.189233e+06
8            8   8  612684.883012  1.189243e+06
9            9   9  612684.883012  1.189253e+06
10          10  10  612684.883012  1.189263e+06
11          11  11  612684.883012  1.189273e+06
12          12  12  612684.883012  1.189283e+06
13          13  13  612684.883012  1.189293e+06
14          14  14  612684.883012  1.189303e+06
15          15  15  612684.883012  1.189313e+06
16          16  16  612684.883012  1.189323e+06
17          17  17  612684.883012  1.189333e+06
18          18  18  612684.883012  1.189343e+06
19          19  19  612684.883012  1.189

In [20]:
# Sắp xếp lại các cột theo thứ tự chieudai, frompoint, topoint
df_sorted = merged_df[['Id', 'StartPoint', 'XX', 'YY']]

print(df_sorted)

    Id  StartPoint             XX            YY
0    0           0  612684.883012  1.189163e+06
1    1           1  612684.883012  1.189173e+06
2    2           2  612684.883012  1.189183e+06
3    3           3  612684.883012  1.189193e+06
4    4           4  612684.883012  1.189203e+06
5    5           5  612684.883012  1.189213e+06
6    6           6  612684.883012  1.189223e+06
7    7           7  612684.883012  1.189233e+06
8    8           8  612684.883012  1.189243e+06
9    9           9  612684.883012  1.189253e+06
10  10          10  612684.883012  1.189263e+06
11  11          11  612684.883012  1.189273e+06
12  12          12  612684.883012  1.189283e+06
13  13          13  612684.883012  1.189293e+06
14  14          14  612684.883012  1.189303e+06
15  15          15  612684.883012  1.189313e+06
16  16          16  612684.883012  1.189323e+06
17  17          17  612684.883012  1.189333e+06
18  18          18  612684.883012  1.189343e+06
19  19          19  612684.883012  1.189

In [21]:
# # Đổi tên các cột
df = df_sorted.rename(columns={"StartPoint": "FacilityPoints"})
print(df)

    Id  FacilityPoints             XX            YY
0    0               0  612684.883012  1.189163e+06
1    1               1  612684.883012  1.189173e+06
2    2               2  612684.883012  1.189183e+06
3    3               3  612684.883012  1.189193e+06
4    4               4  612684.883012  1.189203e+06
5    5               5  612684.883012  1.189213e+06
6    6               6  612684.883012  1.189223e+06
7    7               7  612684.883012  1.189233e+06
8    8               8  612684.883012  1.189243e+06
9    9               9  612684.883012  1.189253e+06
10  10              10  612684.883012  1.189263e+06
11  11              11  612684.883012  1.189273e+06
12  12              12  612684.883012  1.189283e+06
13  13              13  612684.883012  1.189293e+06
14  14              14  612684.883012  1.189303e+06
15  15              15  612684.883012  1.189313e+06
16  16              16  612684.883012  1.189323e+06
17  17              17  612684.883012  1.189333e+06
18  18      

In [22]:
import os

# Đường dẫn đến thư mục
folder_path = r'C:/Users/ASUS/Downloads/Graph/DB_Result_CSV_Rescue_Station'

# Kiểm tra nếu thư mục không tồn tại, tạo mới
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Tạo đường dẫn đến file CSV trong thư mục
file_path = os.path.join(folder_path, 'facility_points.csv')

# Lưu dataframe thành file CSV
df.to_csv(file_path, index=False)

# Hiển thị kết quả
print("File CSV đã được lưu thành công vào thư mục:", folder_path)

File CSV đã được lưu thành công vào thư mục: C:/Users/ASUS/Downloads/Graph/DB_Result_CSV_Rescue_Station
